## Create Points for StreamStats

__Description__:

__Input__:

__Output__:


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

#### Load libraries and Python options:

In [1]:
import os
import collections 
import pandas as pd
import numpy as np
import geopandas as gpd
from osgeo import gdal, ogr,osr
from shapely.geometry import Point
from StreamStats_Points import*

#### Load the masked stream grid:

In [2]:
in_tif=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was masked by the catchment polygon

In [3]:
sg = StreamGrid(in_tif) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) #Print the value

epsg: 5070


In [4]:
df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Specify the pour point to set the start location of the search:

In [5]:
lat=1925315.186 #latitude of the pourpoint at the catchment outlet
lon=1616784.964 #longitude of the pourpoint

In [6]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print(pourpoint)

[(877, 1848)]


In [7]:
cellsize=sg.cell_size() #In meters
print("The Cell Size:",cellsize)

The Cell Size: 10.0


#### Move up the stream and identify the confluences:

##### Function to be saved:

In [8]:
def MoveUpstream(df, stream_cell, nogo):
    """This function searches the 8 cells surrounding it to determine the location of the next stream cell(s).
        Arguments: df=the dataframe containing the stream raster data, stream_cell=the current stream cell that will be used to search for the next stream cell,
        nogo=list of stream cells that do not want to return as a new stream cell
    """
    row=stream_cell[0] #Extract the raster row 
    col=stream_cell[1] #Extract the raster column
    cell_value= df[row][col] #Determine the value of the cell
    
    assert cell_value == 1, "The provided cell in MoveUpstream is not a stream cell" #The value of the cell must be equal to the value associated with a stream cell
    
    nogo.append((row,col))   

    stream_cell=[]
    
    for i in range(-1,2): #For -1, 0, 1 in the vertical direction (move up and down rows)
        for j in range(-1,2): #For -1, 0, 1 in the horizontal direction (move across columns)
            value = df[row + i][col+j] #Read value of raster cell
            if value == 0: # if value is zero, no stream in this cell
                continue #loop back
            elif value==1 and (row+i,col+j) not in nogo: # if value is 1 and the cell is not in the nogo list, then...
                stream_cell.append((row + i,col+j)) #Add the new cell or cells to the stream_cell list
                
    return stream_cell, nogo    

In [86]:
def FindConfluence(df, stream_cell, nogo, mainstem, dis, DisExl): #Keep moving upstream until you find a confluence--problem: might hit a dead end, maybe add an else if it is the end?
    """Function which repeates the MoveUpstream function until it finds two or more stream cells surrounding the provided stream_cell, indicating a confluence
    """
    assert len(stream_cell)==1, "Multiple stream cells passed to FindConfluence Function"
    
    while len(stream_cell)==1: #While there is only 1 stream cell returned as we move up stream, keep moving up stream
        cell1=stream_cell[0]
        stream_cell, nogo=MoveUpstream(df, stream_cell[0], nogo)
        if len(stream_cell)>=1:
            cell2=stream_cell[0]  
            cell_dis=TrueDistance(cell1, cell2, cellsize)
            dis=dis+cell_dis
            dis_lower=dis/DisExl-(10.0*np.sqrt(2))/DisExl
            dis_upper=dis/DisExl+(10.0*np.sqrt(2))/DisExl  
            if dis_lower<=int(dis/DisExl)<=dis_upper and cell1 not in mainstem:
                mainstem.append(stream_cell[0])
    if len(stream_cell)>1: #If the number of stream cells 
        nogo=list(set(nogo+stream_cell))
        
    return stream_cell, nogo, mainstem, dis

In [96]:
def CheckTribDis(stream_cell, nogo, dis, DisExl):
    """Check to see if the tributary has a confluence, if it does not and is not long enough, then remove it 
    """
    nogo_temp=nogo.copy()
    mainstem_temp=[]
    remove=[]

    for cell in stream_cell:
        dis_temp=dis.copy()
        stream_cell1, nogo_temp, mainstem_temp, dis_temp=FindConfluence(df, [cell], nogo_temp, mainstem_temp, dis_temp, DisExl)
        if len(stream_cell1)==0 and dis_temp<DisExl:
            remove.append(cell)
            
    stream_cell=list(set(stream_cell) - set(remove))
    return stream_cell

In [97]:
def CheckConfluence(df, stream_cell, nogo, mainstem, dis, DisExl):
    """
    """
    assert len(stream_cell)==1, "Multiple stream cells passed to CheckConfluence Function"
    
    while len(stream_cell)==1:
        stream_cell, nogo, mainstem, dis=FindConfluence(df, stream_cell, nogo, mainstem, dis, DisExl)
        if len(stream_cell)>1:
            stream_cell=CheckTribDis(stream_cell, nogo, dis, DisExl)
        #if len==0? Then end of network?
        
    return stream_cell, nogo, mainstem, dis

In [98]:
def AddConfluencetoDict(df, confl, nogo, mainstem, dis, DisExl, Incl_Confl):
    """
    """
    IC_key=max(list(Incl_Confl.keys()))+1
    dis_confl=dis.copy()

    for i in range(len(confl)):
        dis_temp=dis.copy()
        mainstem_temp=[]
        
        stream_cell, nogo, mainstem_temp, dis_temp=CheckConfluence(df, [confl[i]], nogo, mainstem_temp, dis_temp, DisExl)
        
        if len(stream_cell)==0 and (dis_temp-dis_confl)>=DisExl:
            Incl_Confl[IC_key]={'cell':confl, 'dis':dis_confl}
        if len(stream_cell)>1:
            dis_out=dis_temp.copy()
            next_confl=stream_cell
            mainstem=mainstem+mainstem_temp
    return next_confl, nogo, mainstem, dis_out, Incl_Confl

##### Set parameters and intalize objects

In [103]:
nogo=[] #Empty list to store the nogo points, i.e. the stream cells we previously searched
mainstem=[]

dis=0.0
DisExl=(5280/2.0)/3.28084 #1/2 mile in In meters

Incl_Confl=collections.OrderedDict()
Incl_Confl[0]={'cell':pourpoint, 'dis':dis}

##### Find the initial confluence:

In [104]:
stream_cell=pourpoint

confl, nogo, mainstem, dis=CheckConfluence(df, stream_cell, nogo, mainstem, dis, DisExl) 

print(confl, dis)

[(880, 1786), (879, 1786)] 757.6955262170047


##### Determine if the initial confluence should be saved, find the next confluence, and repeat:

In [105]:
confl, nogo, mainstem, dis, Incl_confl=AddConfluencetoDict(df, confl, nogo, mainstem, dis, DisExl, Incl_Confl)
print(confl, dis)

#####Comeback to issue with distance up tributaries with multiple points. tried adding dis to checktrib but may have messed up

[(893, 1742), (892, 1742)] 1247.4011537017768


In [81]:
IC_key=max(list(Incl_Confl.keys()))+1
dis_confl=dis.copy()

i=1
dis_temp=dis.copy()
mainstem_temp=[]

In [84]:
stream_cell, nogo, mainstem, dis_temp=FindConfluence(df, [confl[i]], nogo, mainstem, dis_temp, DisExl)

In [85]:
print(stream_cell, dis_temp-dis_confl)

[(836, 1585), (836, 1586), (837, 1584)] 461.1269837220834


In [53]:
stream_cell=CheckTribDis(stream_cell, nogo, DisExl)


In [53]:
        
if len(stream_cell)==0 and (dis_temp-dis_confl)>=DisExl:
    Incl_Confl[IC_key]={'cell':confl, 'dis':dis_confl}
if len(stream_cell)>1:
    dis_out=dis_temp.copy()
    next_confl=stream_cell
    mainstem=mainstem+mainstem_temp

In [15]:
while len(confl)>1:
    confl, nogo, mainstem, dis, Incl_confl=AddConfluencetoDict(df, confl, nogo, mainstem, dis, DisExl, Incl_Confl)

UnboundLocalError: local variable 'next_confl' referenced before assignment

#### Save the results:

##### Save the stream cells to a list:

In [17]:
confl_lst=[]
dist_lst=[]

for v in Incl_Confl.values():
    confl_lst=confl_lst+v['cell']
    dist_lst=dist_lst+list(np.ones(len(v['cell']))*v['dis'])

##### Save the list of stream cells to a shapefile:

In [18]:
longitude, latitude=index2coord(sg, confl_lst)
gdf=geodataframe(longitude, latitude, crs, dist_lst)
gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\Confl.shp')

longitude, latitude=index2coord(sg, mainstem)
gdf=geodataframe(longitude, latitude, crs)
gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\mainstem.shp')

C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# End